In [ ]:
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QLabel, QLineEdit,
    QPushButton, QMessageBox, QInputDialog
)

# Função de login
def login(usuario, senha):
    try:
        alunos = pd.read_excel(r"C:\Users\suporteti\Downloads\alunos.xlsx")
        professores = pd.read_excel(r"C:\Users\suporteti\Downloads\professores.xlsx")
    except FileNotFoundError:
        QMessageBox.critical(None, "Erro", "Arquivos de usuários não encontrados.")
        return None

    alunos['usuario'] = alunos['usuario'].astype(str).str.strip()
    alunos['senha'] = alunos['senha'].astype(str).str.strip()

    professores['usuario'] = professores['usuario'].astype(str).str.strip()
    professores['senha'] = professores['senha'].astype(str).str.strip()

    aluno = alunos[(alunos['usuario'] == usuario) & (alunos['senha'] == senha)]
    if not aluno.empty:
        QMessageBox.information(None, "Login", f"Login realizado como Aluno: {aluno['nome'].values[0]}")
        return "aluno"
    
    professor = professores[(professores['usuario'] == usuario) & (professores['senha'] == senha)]
    if not professor.empty:
        QMessageBox.information(None, "Login", f"Login realizado como Professor: {professor['nome'].values[0]}")
        return "professor"
    
    QMessageBox.warning(None, "Erro", "Login inválido. Usuário ou senha incorretos.")
    return None

# Classe para o menu do Aluno
class MenuAluno(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Menu Aluno")
        layout = QVBoxLayout()

        layout.addWidget(QPushButton("Consultar Notas", clicked=self.consultar_notas))
        layout.addWidget(QPushButton("Verificar Faltas", clicked=self.verificar_faltas))
        layout.addWidget(QPushButton("Consultar Advertências", clicked=self.consultar_advertencias))
        layout.addWidget(QPushButton("Voltar", clicked=self.voltar))

        self.setLayout(layout)

    def consultar_notas(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Notas", "Digite sua matrícula:")
        if not ok:
            return

        try:
            notas = pd.read_excel(r"C:\Users\suporteti\Downloads\notas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de notas não encontrado.")
            return

        boletim = notas[notas['matricula'].astype(str) == matricula]
        if boletim.empty:
            QMessageBox.information(self, "Consultar Notas", "Nenhuma nota encontrada para a matrícula.")
        else:
            QMessageBox.information(self, "Boletim", boletim.to_string())

    def verificar_faltas(self):
        matricula, ok = QInputDialog.getText(self, "Verificar Faltas", "Digite sua matrícula:")
        if not ok:
            return

        try:
            faltas = pd.read_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de faltas não encontrado.")
            return

        faltas_aluno = faltas[faltas['matricula'].astype(str) == matricula]
        if faltas_aluno.empty:
            QMessageBox.information(self, "Verificar Faltas", "Nenhuma falta registrada para a matrícula.")
        else:
            QMessageBox.information(self, "Faltas", faltas_aluno.to_string())

    def consultar_advertencias(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Advertências", "Digite sua matrícula:")
        if not ok:
            return

        try:
            advertencias = pd.read_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de advertências não encontrado.")
            return

        advertencias_aluno = advertencias[advertencias['matricula'].astype(str) == matricula]
        if advertencias_aluno.empty:
            QMessageBox.information(self, "Consultar Advertências", "Nenhuma advertência encontrada para a matrícula.")
        else:
            QMessageBox.information(self, "Advertências", advertencias_aluno.to_string())

    def voltar(self):
        self.close()
        self.login_window = LoginWindow()
        self.login_window.show()

# Classe para o menu do Professor
class MenuProfessor(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Menu Professor")
        layout = QVBoxLayout()

        layout.addWidget(QPushButton("Cadastrar Notas", clicked=self.cadastrar_notas))
        layout.addWidget(QPushButton("Registrar Faltas", clicked=self.registrar_faltas))
        layout.addWidget(QPushButton("Atualizar Grade", clicked=self.atualizar_grade))
        layout.addWidget(QPushButton("Registrar Advertência", clicked=self.registrar_advertencia))
        layout.addWidget(QPushButton("Consultar Notas Cadastradas", clicked=self.consultar_notas))
        layout.addWidget(QPushButton("Consultar Faltas Registradas", clicked=self.consultar_faltas))
        layout.addWidget(QPushButton("Consultar Advertências Aplicadas", clicked=self.consultar_advertencias))
        layout.addWidget(QPushButton("Voltar", clicked=self.voltar))

        self.setLayout(layout)

    def cadastrar_notas(self):
        matricula, ok = QInputDialog.getText(self, "Cadastrar Notas", "Digite a matrícula:")
        if not ok:
            return

        disciplina, _ = QInputDialog.getText(self, "Cadastrar Notas", "Digite a disciplina:")
        nota, _ = QInputDialog.getDouble(self, "Cadastrar Notas", "Digite a nota:", 0, 0, 10, 2)

        try:
            notas = pd.read_excel(r"C:\Users\suporteti\Downloads\notas.xlsx")
        except FileNotFoundError:
            notas = pd.DataFrame(columns=['matricula', 'disciplina', 'nota'])

        notas = pd.concat([notas, pd.DataFrame([{'matricula': matricula, 'disciplina': disciplina, 'nota': nota}])], ignore_index=True)
        notas.to_excel(r"C:\Users\suporteti\Downloads\notas.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Nota cadastrada com sucesso.")

    def registrar_faltas(self):
        matricula, ok = QInputDialog.getText(self, "Registrar Faltas", "Digite a matrícula:")
        if not ok:
            return

        faltas, _ = QInputDialog.getInt(self, "Registrar Faltas", "Digite o número de faltas:")

        try:
            data = pd.read_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx")
        except FileNotFoundError:
            data = pd.DataFrame(columns=['matricula', 'faltas'])

        data = pd.concat([data, pd.DataFrame([{'matricula': matricula, 'faltas': faltas}])], ignore_index=True)
        data.to_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Faltas registradas com sucesso.")

    def atualizar_grade(self):
        turma, ok = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite a turma:")
        if not ok:
            return

        disciplina, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite a disciplina:")
        dia, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite o dia da semana:")
        horario, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite o horário:")
        professor, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite o nome do professor:")

        try:
            grade_aulas = pd.read_excel(r"C:\Users\suporteti\Downloads\grade_aulas.xlsx")
        except FileNotFoundError:
            grade_aulas = pd.DataFrame(columns=['turma', 'disciplina', 'dia', 'horario', 'professor'])

        nova_grade = {'turma': turma, 'disciplina': disciplina, 'dia': dia, 'horario': horario, 'professor': professor}
        grade_aulas = pd.concat([grade_aulas, pd.DataFrame([nova_grade])], ignore_index=True)
        grade_aulas.to_excel(r"C:\Users\suporteti\Downloads\grade_aulas.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Grade de aulas atualizada com sucesso.")

    def registrar_advertencia(self):
        matricula, ok = QInputDialog.getText(self, "Registrar Advertência", "Digite a matrícula do aluno:")
        if not ok:
            return

        motivo, _ = QInputDialog.getText(self, "Registrar Advertência", "Digite o motivo da advertência:")
        descricao, _ = QInputDialog.getText(self, "Registrar Advertência", "Digite a descrição da advertência:")

        try:
            advertencias = pd.read_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx")
        except FileNotFoundError:
            advertencias = pd.DataFrame(columns=['matricula', 'motivo', 'descricao', 'data'])

        data = pd.Timestamp.now().strftime('%d/%m/%Y')
        advertencias = pd.concat([advertencias, pd.DataFrame([{'matricula': matricula, 'motivo': motivo, 'descricao': descricao, 'data': data}])], ignore_index=True)
        advertencias.to_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Advertência registrada com sucesso.")

    def consultar_notas(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Notas", "Digite a matrícula do aluno:")
        if not ok:
            return

        try:
            notas = pd.read_excel(r"C:\Users\suporteti\Downloads\notas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de notas não encontrado.")
            return

        boletim = notas[notas['matricula'].astype(str) == matricula]
        if boletim.empty:
            QMessageBox.information(self, "Consultar Notas", "Nenhuma nota encontrada para a matrícula.")
        else:
            QMessageBox.information(self, "Boletim", boletim.to_string())

    def consultar_faltas(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Faltas", "Digite a matrícula do aluno:")
        if not ok:
            return

        try:
            faltas = pd.read_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de faltas não encontrado.")
            return

        faltas_aluno = faltas[faltas['matricula'].astype(str) == matricula]
        if faltas_aluno.empty:
            QMessageBox.information(self, "Consultar Faltas", "Nenhuma falta registrada para a matrícula.")
        else:
            QMessageBox.information(self, "Faltas", faltas_aluno.to_string())

    def consultar_advertencias(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Advertências", "Digite a matrícula do aluno:")
        if not ok:
            return

        try:
            advertencias = pd.read_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de advertências não encontrado.")
            return

        advertencias_aluno = advertencias[advertencias['matricula'].astype(str) == matricula]
        if advertencias_aluno.empty:
            QMessageBox.information(self, "Consultar Advertências", "Nenhuma advertência registrada para a matrícula.")
        else:
            QMessageBox.information(self, "Advertências", advertencias_aluno.to_string())

    def voltar(self):
        self.close()
        self.login_window = LoginWindow()
        self.login_window.show()

# Classe de Login
class LoginWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Login")
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        layout = QVBoxLayout()

        self.usuario_input = QLineEdit()
        self.usuario_input.setPlaceholderText("Usuário")
        layout.addWidget(self.usuario_input)

        self.senha_input = QLineEdit()
        self.senha_input.setEchoMode(QLineEdit.Password)
        self.senha_input.setPlaceholderText("Senha")
        layout.addWidget(self.senha_input)

        login_button = QPushButton("Login")
        login_button.clicked.connect(self.fazer_login)
        layout.addWidget(login_button)

        central_widget.setLayout(layout)

    def fazer_login(self):
        usuario = self.usuario_input.text()
        senha = self.senha_input.text()
        tipo_usuario = login(usuario, senha)

        if tipo_usuario == "professor":
            self.menu_professor = MenuProfessor()
            self.menu_professor.show()
            self.close()
        elif tipo_usuario == "aluno":
            self.menu_aluno = MenuAluno()
            self.menu_aluno.show()
            self.close()

# Inicialização da aplicação
if __name__ == "__main__":
    app = QApplication([])
    window = LoginWindow()
    window.show()
    app.exec_()


In [11]:
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QLabel, QLineEdit,
    QPushButton, QMessageBox, QInputDialog
)

# Função de login
def login(usuario, senha):
    try:
        alunos = pd.read_excel(r"C:\Users\suporteti\Downloads\alunos.xlsx")
        professores = pd.read_excel(r"C:\Users\suporteti\Downloads\professores.xlsx")
    except FileNotFoundError:
        QMessageBox.critical(None, "Erro", "Arquivos de usuários não encontrados.")
        return None

    alunos['usuario'] = alunos['usuario'].astype(str).str.strip()
    alunos['senha'] = alunos['senha'].astype(str).str.strip()

    professores['usuario'] = professores['usuario'].astype(str).str.strip()
    professores['senha'] = professores['senha'].astype(str).str.strip()

    aluno = alunos[(alunos['usuario'] == usuario) & (alunos['senha'] == senha)]
    if not aluno.empty:
        QMessageBox.information(None, "Login", f"Login realizado como Aluno: {aluno['nome'].values[0]}")
        return "aluno"
    
    professor = professores[(professores['usuario'] == usuario) & (professores['senha'] == senha)]
    if not professor.empty:
        nome_professor = professor['nome'].values[0]
        disciplina_professor = professor['disciplina'].values[0]
        QMessageBox.information(None, "Login", f"Login realizado como Professor: {nome_professor}\nDisciplina: {disciplina_professor}")
        return "professor"
    
    QMessageBox.warning(None, "Erro", "Login inválido. Usuário ou senha incorretos.")
    return None

# Classe para o menu do Aluno
class MenuAluno(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Menu Aluno")
        layout = QVBoxLayout()

        layout.addWidget(QPushButton("Consultar Notas", clicked=self.consultar_notas))
        layout.addWidget(QPushButton("Verificar Faltas", clicked=self.verificar_faltas))
        layout.addWidget(QPushButton("Consultar Advertências", clicked=self.consultar_advertencias))
        layout.addWidget(QPushButton("Voltar", clicked=self.voltar))

        self.setLayout(layout)

    def consultar_notas(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Notas", "Digite sua matrícula:")
        if not ok:
            return

        try:
            notas = pd.read_excel(r"C:\Users\suporteti\Downloads\notas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de notas não encontrado.")
            return

        boletim = notas[notas['matricula'].astype(str) == matricula]
        if boletim.empty:
            QMessageBox.information(self, "Consultar Notas", "Nenhuma nota encontrada para a matrícula.")
        else:
            QMessageBox.information(self, "Boletim", boletim.to_string())

    def verificar_faltas(self):
        matricula, ok = QInputDialog.getText(self, "Verificar Faltas", "Digite sua matrícula:")
        if not ok:
            return

        try:
            faltas = pd.read_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de faltas não encontrado.")
            return

        faltas_aluno = faltas[faltas['matricula'].astype(str) == matricula]
        if faltas_aluno.empty:
            QMessageBox.information(self, "Verificar Faltas", "Nenhuma falta registrada para a matrícula.")
        else:
            QMessageBox.information(self, "Faltas", faltas_aluno.to_string())

    def consultar_advertencias(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Advertências", "Digite sua matrícula:")
        if not ok:
            return

        try:
            advertencias = pd.read_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de advertências não encontrado.")
            return

        advertencias_aluno = advertencias[advertencias['matricula'].astype(str) == matricula]
        if advertencias_aluno.empty:
            QMessageBox.information(self, "Consultar Advertências", "Nenhuma advertência encontrada para a matrícula.")
        else:
            QMessageBox.information(self, "Advertências", advertencias_aluno.to_string())

    def voltar(self):
        self.close()
        self.login_window = LoginWindow()
        self.login_window.show()

# Classe para o menu do Professor
class MenuProfessor(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Menu Professor")
        layout = QVBoxLayout()

        layout.addWidget(QPushButton("Cadastrar Notas", clicked=self.cadastrar_notas))
        layout.addWidget(QPushButton("Registrar Faltas", clicked=self.registrar_faltas))
        layout.addWidget(QPushButton("Atualizar Grade", clicked=self.atualizar_grade))
        layout.addWidget(QPushButton("Registrar Advertência", clicked=self.registrar_advertencia))
        layout.addWidget(QPushButton("Consultar Notas Cadastradas", clicked=self.consultar_notas))
        layout.addWidget(QPushButton("Consultar Faltas Registradas", clicked=self.consultar_faltas))
        layout.addWidget(QPushButton("Consultar Advertências Aplicadas", clicked=self.consultar_advertencias))
        layout.addWidget(QPushButton("Voltar", clicked=self.voltar))

        self.setLayout(layout)

    def cadastrar_notas(self):
        # Obter matrícula e disciplina do professor
        usuario = self.usuario_input.text()  # Assumindo que já temos o usuário logado
        disciplina, _ = QInputDialog.getText(self, "Cadastrar Notas", "Digite a disciplina:")

        # Verificar se a disciplina está associada ao professor
        try:
            professores = pd.read_excel(r"C:\Users\suporteti\Downloads\professores.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de professores não encontrado.")
            return

        professor = professores[professores['usuario'] == usuario]
        if professor.empty:
            QMessageBox.warning(self, "Erro", "Professor não encontrado.")
            return

        disciplina_professor = professor['disciplina'].values[0]
        
        if disciplina != disciplina_professor:
            QMessageBox.warning(self, "Erro", "Você não pode cadastrar notas para essa disciplina.")
            return

        # Continuar com o processo de cadastro de notas...
        matricula, ok = QInputDialog.getText(self, "Cadastrar Notas", "Digite a matrícula do aluno:")
        if not ok:
            return
        
        nota, _ = QInputDialog.getDouble(self, "Cadastrar Notas", "Digite a nota:", 0, 0, 10, 2)
        
        try:
            notas = pd.read_excel(r"C:\Users\suporteti\Downloads\notas.xlsx")
        except FileNotFoundError:
            notas = pd.DataFrame(columns=['matricula', 'disciplina', 'nota'])

        notas = pd.concat([notas, pd.DataFrame([{'matricula': matricula, 'disciplina': disciplina, 'nota': nota}])], ignore_index=True)
        notas.to_excel(r"C:\Users\suporteti\Downloads\notas.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Nota cadastrada com sucesso.")

    def registrar_faltas(self):
        matricula, ok = QInputDialog.getText(self, "Registrar Faltas", "Digite a matrícula:")
        if not ok:
            return

        faltas, _ = QInputDialog.getInt(self, "Registrar Faltas", "Digite o número de faltas:")

        try:
            data = pd.read_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx")
        except FileNotFoundError:
            data = pd.DataFrame(columns=['matricula', 'faltas'])

        data = pd.concat([data, pd.DataFrame([{'matricula': matricula, 'faltas': faltas}])], ignore_index=True)
        data.to_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Faltas registradas com sucesso.")

    def atualizar_grade(self):
        turma, ok = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite a turma:")
        if not ok:
            return

        disciplina, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite a disciplina:")
        dia, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite o dia da semana:")
        horario, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite o horário:")
        professor, _ = QInputDialog.getText(self, "Atualizar Grade de Aulas", "Digite o nome do professor:")

        try:
            grade_aulas = pd.read_excel(r"C:\Users\suporteti\Downloads\grade_aulas.xlsx")
        except FileNotFoundError:
            grade_aulas = pd.DataFrame(columns=['turma', 'disciplina', 'dia', 'horario', 'professor'])

        grade_aulas = pd.concat([grade_aulas, pd.DataFrame([{'turma': turma, 'disciplina': disciplina, 'dia': dia, 'horario': horario, 'professor': professor}])], ignore_index=True)
        grade_aulas.to_excel(r"C:\Users\suporteti\Downloads\grade_aulas.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Grade de aulas atualizada com sucesso.")

    def registrar_advertencia(self):
        matricula, ok = QInputDialog.getText(self, "Registrar Advertência", "Digite a matrícula do aluno:")
        if not ok:
            return
        
        advertencia, _ = QInputDialog.getText(self, "Registrar Advertência", "Digite a advertência:")

        try:
            advertencias = pd.read_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx")
        except FileNotFoundError:
            advertencias = pd.DataFrame(columns=['matricula', 'advertencia'])

        advertencias = pd.concat([advertencias, pd.DataFrame([{'matricula': matricula, 'advertencia': advertencia}])], ignore_index=True)
        advertencias.to_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx", index=False)
        QMessageBox.information(self, "Sucesso", "Advertência registrada com sucesso.")

    def consultar_notas(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Notas", "Digite a matrícula do aluno:")
        if not ok:
            return

        try:
            notas = pd.read_excel(r"C:\Users\suporteti\Downloads\notas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de notas não encontrado.")
            return

        boletim = notas[notas['matricula'].astype(str) == matricula]
        if boletim.empty:
            QMessageBox.information(self, "Consultar Notas", "Nenhuma nota encontrada para a matrícula.")
        else:
            QMessageBox.information(self, "Boletim", boletim.to_string())

    def consultar_faltas(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Faltas", "Digite a matrícula do aluno:")
        if not ok:
            return

        try:
            faltas = pd.read_excel(r"C:\Users\suporteti\Downloads\faltas.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de faltas não encontrado.")
            return

        faltas_aluno = faltas[faltas['matricula'].astype(str) == matricula]
        if faltas_aluno.empty:
            QMessageBox.information(self, "Consultar Faltas", "Nenhuma falta registrada para a matrícula.")
        else:
            QMessageBox.information(self, "Faltas", faltas_aluno.to_string())

    def consultar_advertencias(self):
        matricula, ok = QInputDialog.getText(self, "Consultar Advertências", "Digite a matrícula do aluno:")
        if not ok:
            return

        try:
            advertencias = pd.read_excel(r"C:\Users\suporteti\Downloads\advertencias.xlsx")
        except FileNotFoundError:
            QMessageBox.critical(self, "Erro", "Arquivo de advertências não encontrado.")
            return

        advertencias_aluno = advertencias[advertencias['matricula'].astype(str) == matricula]
        if advertencias_aluno.empty:
            QMessageBox.information(self, "Consultar Advertências", "Nenhuma advertência encontrada para a matrícula.")
        else:
            QMessageBox.information(self, "Advertências", advertencias_aluno.to_string())

    def voltar(self):
        self.close()
        self.login_window = LoginWindow()
        self.login_window.show()

# Classe principal da janela de login
class LoginWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Login")
        self.setGeometry(100, 100, 280, 120)

        layout = QVBoxLayout()

        self.usuario_input = QLineEdit(self)
        self.usuario_input.setPlaceholderText("Usuário")
        layout.addWidget(self.usuario_input)

        self.senha_input = QLineEdit(self)
        self.senha_input.setEchoMode(QLineEdit.Password)
        self.senha_input.setPlaceholderText("Senha")
        layout.addWidget(self.senha_input)

        login_button = QPushButton("Login", self)
        login_button.clicked.connect(self.logar)
        layout.addWidget(login_button)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def logar(self):
        usuario = self.usuario_input.text()
        senha = self.senha_input.text()

        tipo_usuario = login(usuario, senha)

        if tipo_usuario == "aluno":
            self.menu_aluno = MenuAluno()
            self.menu_aluno.show()
            self.close()
        elif tipo_usuario == "professor":
            self.menu_professor = MenuProfessor()
            self.menu_professor.show()
            self.close()
